<h2>import data from excel</h2>

In [15]:
from gurobipy import*
import math
import pandas as pd
import numpy as np
import csv

In [33]:
df={}
Demand_num=100 #follow the input file
Demand = list()
Demand = list(range(1,Demand_num+1))
df=pd.read_csv("./prj1_test_data/Car_Demand_"+str(Demand_num)+".csv")
Type_d={}
Location_d={}
Time={}
Penalty={}
for i in range(1,Demand_num+1):
    Location_d[i] = list()
    Time[i] = list()
    Type_d[i]=df.iloc[i-1]["type"]
    Location_d[i].append(df.iloc[i-1]["Location_X"])
    Location_d[i].append(df.iloc[i-1]["Location_Y"])
    Time[i].append(df.iloc[i-1]["Time_H"])
    Time[i].append(df.iloc[i-1]["Time_E"])
    Penalty[i]=df.iloc[i-1]["Penalty"]


In [34]:
# print(Demand)
# print()
# print(Type_d)
# print()
# print(Location_d)
# print()
# print(Time)
# print()
# print(Penalty)

In [35]:
df2={}
Supply_num=200
Supply = list()
Supply = list(range(1,Supply_num+1))
df2=pd.read_csv("./prj1_test_data/Space_"+str(Supply_num)+".csv")
Type_s={}
Location_s={}
for i in range(1,Supply_num+1):
    Location_s[i]=list()
    Type_s[i]=df2.iloc[i-1]["type"]
    Location_s[i].append(df2.iloc[i-1]["Location_X"])
    Location_s[i].append(df2.iloc[i-1]["Location_Y"])

In [36]:
# print(Supply)
# print(Type_s)
# print(Location_s)

In [37]:
def check_time_overlap(a,b):
    if (a[0]<b[0]):
        if (b[0]-a[0]) < (a[1] - a[0]):
            return True
        else:
            return False
    elif (a[0]>b[0]):
        if (a[0]-b[0]) < (b[1] - b[0]):
            return True
        else:
            return False
    else :
        return True
        
def distance(a,b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)


In [39]:
# print(check_time_overlap([100,150],[0,100]))
# print(distance([45,8],[30,33]))

In [40]:
model = Model("Car_parking")
x={}
y={}
b={}

<h2>init var</h2>

In [41]:
for d in Demand:
    y[d] = model.addVar(vtype = "B",name = "y(%s)"%d)
    for s in Supply:
        x[d,s] = model.addVar(vtype="B",name = "x(%s,%s)"%(d,s))
        b[d,s] = model.addVar(lb = 0, vtype="C", name = "b(%s,%s)"%(d,s))
model.update()

In [42]:
# print(Demand)
# print()
# print(Supply)
# print()
# print(Type_d)
# print()
# print(Type_s)
# print()
# print(x)
# print()
# print(y)
# print()
# print(b)

<h1>add constraint</h1>
<h2>time constraint</h2>
<br>
<h3>one demand correspond to one supply</h3>
<h3>But a supply can correspond to several demand</h3>
<p>quicksum(x[d,s] for s in Supply) = 1 for d in Demand</p>
<br>
<h3>Type constraint</h3>
   

In [43]:
R=20
#demand constraint one car can at most park in one lot
for d in Demand:
    model.addConstr(quicksum(x[d,s] for s in Supply) <= 1,name = "Demand constraint")
    
#time constraint
for i in Demand:
    for j in range(i+1,Demand_num+1):
        if check_time_overlap(Time[i],Time[j]) == True:
            for s in Supply:
                model.addConstr((x[i,s]+x[j,s]) <= 1, name = "Time overlap constraint" ) 
                
#type constraint
for d in Demand:
    for s in Supply:
        if Type_d[d] > Type_s[s]:
            model.addConstr(x[d,s] == 0, name = "Type constraint")
            
#distance constriant 
for d in Demand:
    for s in Supply:
        if distance(Location_d[d],Location_s[s]) > 20:
            model.addConstr(x[d,s] == 0, name = "distance constraint")


#penalty contraint sum(x[d,s] for Supply)+y ==1
for d in Demand:
    model.addConstr(quicksum(x[d,s] for s in Supply) + y[d] == 1, name = "penalty constraint")

#benefit constraint b[d,s] = r=math.sqrt((Location_d[d][0]-Location_s[s][0])**2+(Location_d[d][1]-Location_s[s][1])**2)
for d in Demand:
    for s in Supply:
        radius = distance(Location_d[d],Location_s[s])
        model.addConstr(b[d,s] == (Penalty[d]/radius), name = "benefit define")


In [44]:

model.setObjective(quicksum(b[d,s]*x[d,s] for (d,s) in x)-quicksum(Penalty[d]*y[d] for d in Demand),GRB.MAXIMIZE)

In [45]:
model.optimize()
print("optimize",model.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 765071 rows, 40100 columns and 1529571 nonzeros
Model fingerprint: 0x7a17c203
Model has 20000 quadratic objective terms
Variable types: 20000 continuous, 20100 integer (20100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 5e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-01, 4e+02]
Found heuristic solution: objective -27714.00000
Presolve removed 763637 rows and 35564 columns
Presolve time: 0.90s
Presolved: 1434 rows, 4536 columns, 23502 nonzeros
Variable types: 0 continuous, 4536 integer (4536 binary)

Root relaxation: objective 1.314445e+04, 816 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0          

In [46]:

for d in Demand:
    for s in Supply:
        if x[d,s].X > 0:
            print("car %s park in space %s: %s"%(d,s,x[d,s].X))

car 1 park in space 162: 1.0
car 2 park in space 108: 1.0
car 3 park in space 60: 1.0
car 4 park in space 102: 1.0
car 5 park in space 93: 1.0
car 6 park in space 130: 1.0
car 7 park in space 170: 1.0
car 8 park in space 66: 1.0
car 9 park in space 135: 1.0
car 10 park in space 110: 1.0
car 11 park in space 28: 1.0
car 12 park in space 84: 1.0
car 13 park in space 133: 1.0
car 14 park in space 124: 1.0
car 15 park in space 96: 1.0
car 16 park in space 78: 1.0
car 17 park in space 177: 1.0
car 18 park in space 33: 1.0
car 19 park in space 160: 1.0
car 20 park in space 183: 1.0
car 21 park in space 38: 1.0
car 22 park in space 158: 1.0
car 23 park in space 2: 1.0
car 24 park in space 122: 1.0
car 25 park in space 40: 1.0
car 26 park in space 196: 1.0
car 27 park in space 161: 1.0
car 28 park in space 31: 1.0
car 29 park in space 52: 1.0
car 30 park in space 106: 1.0
car 31 park in space 65: 1.0
car 32 park in space 39: 1.0
car 33 park in space 149: 1.0
car 34 park in space 99: 1.0
car 35

In [29]:
def show_demand(a):
    for i in a:
        print(Type_d[i], Location_d[i], Time[i], Penalty[i])


In [30]:
show_demand([1,5,14])

2 [15, 22] [556, 794] 128
3 [41, 19] [83, 595] 379
3 [48, 34] [330, 575] 484


In [31]:
!cat prj1_test_data/Space_1.csv

In [32]:
!ls prj1_test_data

Car_Demand_10.csv   Car_Demand_20.csv   Space_1.csv         answer_2.docx
Car_Demand_100.csv  Car_Demand_200.csv  Space_10.csv        ~$swer_2.docx
Car_Demand_1000.csv Car_Demand_5.csv    Space_200.csv
